# Keras によるモデル開発 (ローカル環境)

Keras を利用したモデル学習を行います。手書き数字を区別する画像認識モデルを構築します。

## Azure Machine Learning Python SDK インポート

In [1]:
from azureml.core import Workspace, Dataset, Experiment
print(azureml.core.VERSION)

1.0.65


## Azure Machine Learning Workspace への接続
Azure Machine Learning service へ接続を行います。

In [2]:
subscription_id = '9c0f91b8-eb2f-484c-979c-15848c098a6b'
resource_group = 'dllab-test'
workspace_name = 'azureml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

## 実験名の設定
過去の実験履歴をまとめるために名前をつけておきます。

In [3]:
experiment = Experiment(workspace = workspace, name = "keras_mnist_local")

## モデル学習コード

### ライブラリのインポート

In [4]:
from __future__ import print_function
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation
from keras.optimizers import SGD
from keras.utils import np_utils
import tensorflow as tf
#from keras.utils import plot_model
from tensorflow.python.keras.utils.vis_utils import plot_model
#from tensorflow.keras.vis_utils import plot_model

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### ハイパーパラメータ設定

In [5]:
NB_EPOCH = 20
BATCH_SIZE = 32
NB_CLASSES = 10 #number of outputs = number of digits
OPTIMIZER = SGD() #SGD Optimizer
N_HIDDEN = 128
VALIDATION_SPLIT = 0.3 # Training and Validation data ratio
DROPOUT = 0.3 

In [6]:
class RunCallback(tf.keras.callbacks.Callback):
    def __init__(self, run):
        self.run = run
        
    def on_epoch_end(self, batch, logs={}):
        self.run.log(name="training_acc", value=float(logs.get('accuracy')))
        self.run.log(name="validation_acc", value=float(logs.get('val_accuracy')))

### 学習データの準備

In [7]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [8]:
RESHAPED = 784

In [9]:
X_train = X_train.reshape(60000,RESHAPED)
X_test = X_test.reshape(10000,RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [10]:
X_train /= 255
X_test /= 255

In [11]:
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

### モデル実装

In [12]:
model = Sequential()
model.add(Dense(N_HIDDEN, input_shape = (RESHAPED,)))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [13]:
callbacks = list()

In [14]:
model.compile(loss ='categorical_crossentropy', optimizer = OPTIMIZER, metrics=['accuracy'])

### Azure Machine Learning service メトリック & ログ 記録開始

In [15]:
run = experiment.start_logging()

In [16]:
callbacks.append(RunCallback(run))

## モデル学習

In [17]:
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE, 
          epochs=NB_EPOCH, 
          callbacks= callbacks,
          verbose = 1, 
          validation_split=VALIDATION_SPLIT)

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 42000 samples, validate on 18000 samples
Epoch 1/20
42000/42000 [==============================] - 4s 89us/step - loss: 1.0342 - accuracy: 0.6757 - val_loss: 0.4117 - val_accuracy: 0.8903
Epoch 2/20
42000/42000 [==============================] - 4s 85us/step - loss: 0.5050 - accuracy: 0.8484 - val_loss: 0.3143 - val_accuracy: 0.9100
Epoch 3/20
42000/42000 [==============================] - 4s 85us/step - loss: 0.4116 - accuracy: 0.8780 - val_loss: 0.2693 - val_accuracy: 0.9233
Epoch 4/20
42000/42000 [==============================] - 4s 85us/step - loss: 0.3578 - accuracy: 0.8940 - val_loss: 0.2418 - val_accuracy: 0.9293
Epoch 5/20
42000/42000 [==============================] - 4s 85us/step - loss: 0.3230 - accuracy: 0.9057 - val_loss: 0.2218 - val_accuracy: 0.9347
Epoch 6/20
42000/42000 [==============================] - 4s 85us/step - loss: 0.2954 - accuracy: 0.9149 - val_loss: 0.2028 - val_accuracy: 0.9400
Epoch 7/20
42000/42000 [==============================] - 4s 87us/st

In [19]:
score = model.evaluate(X_test, Y_test, verbose= 1)
print("Test score:", score[0])
print("Test accuracy" , score[1])
print("Loss value", score[0])
print("Accuracy value", score[1])

10000/10000 [==============================] - 0s 29us/step
Test score: 0.11154165054019541
Test accuracy 0.9674000144004822
Loss value 0.11154165054019541
Accuracy value 0.9674000144004822


## メトリックの記録

In [20]:
run.log("Loss", score[0])
run.log("Accuracy", score[1])
run.log("BATCH_SIZE", BATCH_SIZE)

## モデル保存

In [21]:
# !pip install pydot
# !pip install pydot-ng
# !pip install graphviz
# !sudo apt-get -y install graphviz

In [22]:
with run:
    plot_model(model, to_file='./outputs/model.png')
    model.save_weights('./outputs/mnist_mlp_weights.h5')

In [24]:
run.complete()

In [25]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
keras_mnist_local,05b997ee-af1f-4dac-84af-260f32b9354b,,Running,Link to Azure Portal,Link to Documentation
